In [1]:
import numpy as np
import cv2
from PIL import Image
import copy
import matplotlib.pyplot as plt
import os
from utils import helper
import pandas as pd

In [2]:
# Create dataset
# Columns: image_path, label_path, bboxes, classes
imlabel_list = helper.imlabel('data/kitti/custom_annotated/annotated_images/', 'data/kitti/custom_annotated/labels/')

In [3]:
imlabel_list[0][0]

'data/kitti/custom_annotated/annotated_images/2-wheeler-1.jpg'

In [28]:
df = pd.DataFrame(columns = ['image_path','label_path','class','bboxes'])
# df = pd.DataFrame()
# df['image_path'] = imlabel_list[:][0]
# df['label_path'] = imlabel_list[:][1]
# df.head()

for item in imlabel_list:
    bboxes = helper.fetch_bboxes(item[1]).tolist()
    classes = helper.fetch_classes(item[1])
    for i,cls_bbox in enumerate(zip(classes,bboxes)):
        # print(cls_bbox)
        #df.loc[i,'image_path'] = item[0]
        #df.loc[i,'label_path'] = item[1]
        #df.loc[i,'class'] = cls_bbox[0]
        
        #df.loc[i,'bboxes'] = cls_bbox[1]
        # df = df.append([item[0],item[1],cls_bbox[0],cls_bbox[1]])
        df = df.append({'image_path':item[0], 'label_path': item[1],
                        'class': cls_bbox[0], 'bboxes': cls_bbox[1]}, ignore_index=True)
df.head()        

,image_path,label_path,class,bboxes
0,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[181.0, 83.0, 343.0, 396.0, 7.748604185489348e..."
1,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[557.0, 47.0, 691.0, 377.0, 7.748604185489348e..."
2,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[322.0, 116.0, 439.0, 348.0, 7.748604185489348..."
3,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[426.0, 122.0, 514.0, 295.0, 7.748604185489348..."
4,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[125.0, 134.0, 167.0, 221.0, 7.748604185489348..."


In [32]:
df['class'].unique()

class_dict = {'2-wheeler': 0, 
              'pedestrian': 1, 'car': 2,
              'dontcare': 4}

df_train['class'] = df_train['class'].apply(lambda x:  class_dict[x])

array(['2-wheeler', 'pedestrian', 'car', 'dontcare'], dtype=object)

In [ ]:
class Kitti:
    
    def init():